In [1]:
import h5py
import iris

/Users/theo/external/miniconda3/py3.7/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
NC_FILE='../data/local.nc'


In [3]:
cubes = iris.load(NC_FILE)
cubes

[<iris 'Cube' of precipitation_flux / (kg m-2 s-1) (time: 10; grid_latitude: 219; grid_longitude: 286)>]

In [4]:
c  = cubes[0]

In [8]:
dir(c.dim_coords[0])

['_DEFAULT_NAME',
 '_MODE_ADD',
 '_MODE_DIV',
 '_MODE_MUL',
 '_MODE_RDIV',
 '_MODE_SUB',
 '_MODE_SYMBOL',
 '__abstractmethods__',
 '__add__',
 '__binary_operator__',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_abc_impl',
 '_as_defn',
 '_attributes',
 '_bounds_dm',
 '_climatological',
 '_discontiguity_in_bounds',
 '_guess_bounds',
 '_new_bounds_requirements',
 '_new_points_requirements',
 '_points_dm',
 '_repr_other_metadata',
 '_sanitise_array',
 '_sanity_check_bounds',
 '_standard_

In [12]:
h5file = h5py.File(NC_FILE,'r')
h5file

<HDF5 file "local.nc" (mode r)>

In [13]:
h5file.visit(lambda x: print(x))

bnds
forecast_period
forecast_period_bnds
forecast_reference_time
grid_latitude
grid_longitude
precipitation_flux
rotated_latitude_longitude
time
time_bnds


In [14]:
ds = h5file['precipitation_flux']
ds

<HDF5 dataset "precipitation_flux": shape (10, 219, 286), type "<f4">

In [15]:
ds.name

'/precipitation_flux'

In [16]:
ds.shape


(10, 219, 286)

In [21]:
s = slice(0,10)
list(range(s.start, s.stop))




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [22]:
ds



<HDF5 dataset "precipitation_flux": shape (10, 219, 286), type "<f4">

In [32]:
display(list(ds.attrs))
display(ds.attrs['cell_methods'])
display(ds.attrs['coordinates'])
display(ds.attrs['um_stash_source'])


['DIMENSION_LIST',
 '_Netcdf4Dimid',
 'standard_name',
 'units',
 'um_stash_source',
 'cell_methods',
 'grid_mapping',
 'coordinates']

b'time: mean (interval: 1 hour)'

b'forecast_period forecast_reference_time'

b'm01s05i216'

In [44]:
import re

In [64]:
def tiledb_name_ds(ds):
    attrs = ['cell_methods','um_stash_source']
    name_items = [ds.name.encode('utf-8')]
    name_items += [ds.attrs.get(attr,b'') for attr in attrs]
    if ds.attrs['coordinates'].decode().find('pressure') >=0:
        name_items += [b"at-pressure"]
    if ds.attrs['coordinates'].decode().find('height') >=0:
        name_items += [b"at-height"]
    return re.sub("[^-A-z0-9_]","",b'__'.join(name_items).decode().replace(' ','-'))

tiledb_name_ds(ds)

'precipitation_flux__time-mean-interval-1-hour__m01s05i216'

In [60]:
[ds.attrs.get(attr,b'') for attr in attrs]

[b'time: mean (interval: 1 hour)', b'm01s05i216']

In [61]:
re.sub("[^-A-z0-9_]","",b'__'.join(name_items).decode().replace(' ','-'))

'precipitation_flux__time-mean-interval-1-hour__m01s05i216__at-pressure__at-height'

In [66]:
ds.attrs['coordinates'].decode()

'forecast_period forecast_reference_time'

In [67]:
ds.shape

(10, 219, 286)

In [74]:
[ds.file[ref[0]].name for ref in ds.attrs['DIMENSION_LIST']]

['/time', '/grid_latitude', '/grid_longitude']

In [76]:
! ls ../data/monthly

apepda.pm51feb.nc apepda.pm51jan.nc apepda.pm51jul.nc


In [2]:
import iris
import os

/Users/theo/external/miniconda3/py3.7/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [10]:
[os.path.abspath(os.path.join('../data/monthly',f)) for f in ["apepda.pm51feb.nc", "apepda.pm51jan.nc", "apepda.pm51jul.nc"]]

['/Users/theo/repos/crd_tiledb_xarray/data/monthly/apepda.pm51feb.nc',
 '/Users/theo/repos/crd_tiledb_xarray/data/monthly/apepda.pm51jan.nc',
 '/Users/theo/repos/crd_tiledb_xarray/data/monthly/apepda.pm51jul.nc']

In [3]:
cubes = iris.load([os.path.join('../data/monthly',f) for f in ["apepda.pm51feb.nc", "apepda.pm51jan.nc", "apepda.pm51jul.nc"]])

In [4]:
cubes

[<iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of air_temperature / (K) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of air_temperature / (K) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of air_temperature / (K) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of air_temperature / (K) (time: 3; pressure: 17; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of cloud_area_fraction / (1) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of geopotential_height / (m) (time: 3; pressure: 17; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of lagrangian_tendency_of_air_pressure / (Pa s-1) (time: 3; pressure: 17; grid_latitude: 218; grid_longitude: 286)>,
<iris 'Cube' of precipitation_flux / (kg m-2 s-1) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of relative_humidity / (%) (time: 3; pressure: 17; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of relative_humidity / (%) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of specific_humidity / (1) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of surface_air_pressure / (Pa) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of surface_downwelling_longwave_flux_in_air / (W m-2) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of surface_downwelling_shortwave_flux_in_air / (W m-2) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of surface_temperature / (K) (time: 3; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of x_wind / (m s-1) (time: 3; grid_latitude: 218; grid_longitude: 286)>,
<iris 'Cube' of x_wind / (m s-1) (time: 3; pressure: 17; grid_latitude: 218; grid_longitude: 286)>,
<iris 'Cube' of y_wind / (m s-1) (time: 3; grid_latitude: 218; grid_longitude: 286)>,
<iris 'Cube' of y_wind / (m s-1) (time: 3; pressure: 17; grid_latitude: 218; grid_longitude: 286)>]

In [5]:
cube = cubes[0]
cube

<iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 3; grid_latitude: 219; grid_longitude: 286)>

In [7]:
for coord in cube.dim_coords:
    

(DimCoord(array([-1042764., -1042056., -1038420.]), bounds=array([[-1043136., -1042392.],
        [-1042392., -1041720.],
        [-1038792., -1038048.]]), standard_name='time', units=Unit('hours since 1970-01-01 00:00:00', calendar='gregorian'), var_name='time'),
 DimCoord(array([ 22.88      ,  22.66      ,  22.439999  ,  22.22      ,
         22.        ,  21.779999  ,  21.56      ,  21.34      ,
         21.119999  ,  20.9       ,  20.679998  ,  20.46      ,
         20.24      ,  20.019999  ,  19.8       ,  19.58      ,
         19.359999  ,  19.14      ,  18.919998  ,  18.699999  ,
         18.48      ,  18.259998  ,  18.039999  ,  17.82      ,
         17.599998  ,  17.38      ,  17.16      ,  16.939999  ,
         16.72      ,  16.5       ,  16.279999  ,  16.06      ,
         15.839999  ,  15.619999  ,  15.4       ,  15.179999  ,
         14.959999  ,  14.739999  ,  14.5199995 ,  14.299999  ,
         14.079999  ,  13.86      ,  13.639999  ,  13.419999  ,
         13.199999  , 

In [ ]:
domains = {}

def dom_name(cube):
    for dom in
    cube.